# 7Problem Statement1: Build a text classification RNN ( RecurrentNeural Network) model using IMDB dataset.Import theIMDBdataset using tensorflow_datasets and perform the following tasks

In [1]:
#importing the datasets
import tensorflow as tf
import tensorflow_datasets as tfds


In [2]:
# Next, we can load the IMDB dataset and shuffle the training data:


(train_data, test_data), info = tfds.load('imdb_reviews/subwords8k', split=(tfds.Split.TRAIN, tfds.Split.TEST), with_info=True, as_supervised=True)
train_batches = train_data.shuffle(1000).padded_batch(32, padded_shapes=([None],[]))
test_batches = test_data.shuffle(1000).padded_batch(32, padded_shapes=([None],[]))


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\Kalpesh__K__\tensorflow_datasets\imdb_reviews\subwords8k\1.0.0.incompleteWJS0HH\imdb_review…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\Kalpesh__K__\tensorflow_datasets\imdb_reviews\subwords8k\1.0.0.incompleteWJS0HH\imdb_review…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\Kalpesh__K__\tensorflow_datasets\imdb_reviews\subwords8k\1.0.0.incompleteWJS0HH\imdb_review…

Dataset imdb_reviews downloaded and prepared to C:\Users\Kalpesh__K__\tensorflow_datasets\imdb_reviews\subwords8k\1.0.0. Subsequent calls will reuse this data.


In [ ]:
#Now, we can encode the text data using the TextVectorization layer:


encoder = info.features['text'].encoder


In [ ]:
#This creates an encoder that can map words to integers and back. We can use this encoder to preprocess the text data:

def encode(text_tensor, label):
  encoded_text = encoder.encode(text_tensor.numpy())
  return encoded_text, label

def encode_map_fn(text, label):
  return tf.py_function(encode, inp=[text, label], Tout=(tf.int64, tf.int64))

train_data = train_batches.map(encode_map_fn)
test_data = test_batches.map(encode_map_fn)


In [ ]:
# Now, we can build a sequential model using tf.keras.Sequential function:

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])



In [ ]:
#Next, we can compile the model:


model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), optimizer=tf.keras.optimizers.Adam(1e-4), metrics=['accuracy'])


In [ ]:
# Now, we can train the model using the training data:

history = model.fit(train_data, epochs=10, validation_data=test_data, validation_steps=30)


In [ ]:
#Finally, we can test the model by passing a sentence and evaluating the output:

sentence = "This movie was great!"
encoded_sentence = encoder.encode(sentence)
model.predict(tf.convert_to_tensor([encoded_sentence]))


# 1Problem Statement2: Sumanth wants to build an flowers image classification CNN model which can take a flower image as input and prints the name of flower.Dataset Description:The dataset consistof 3,700 photos of flowers. The dataset contains five sub-directories, one per class:•daisy•dandelion•roses•sunflowers•tulips

In [ ]:
#importing libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os
import PIL


In [ ]:
data_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = keras.utils.get_file('flower_photos', origin=data_url, untar=True)
data_dir



In [ ]:
#We can now analyze the images present in the dataset using the following code:

batch_size = 32
img_height = 180
img_width = 180

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")


In [ ]:
#We will apply cache(), shuffle(), and prefetch() functions to autotune the data and improve the data loading performance. We will also normalize the data by dividing it with 255 using the following code:

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)


In [ ]:
#Building a sequential CNN model:
num_classes = 5

model = keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='


In [ ]:
#Compiling the model and printing the model summary:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()


In [ ]:
#We will train and validate the model using the training and validation datasets, and plot the validation results using a line plot using the following code:

epochs = 10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

# Plotting the validation results
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()


In [ ]:
#We will feed the system with an image of a flower and print the prediction along with the accuracy using the following code:

img = keras.preprocessing.image.load_img(
    "flower.jpg", target_size=(img_height, img_width)
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)


In [ ]:
# Prepare the data by specifying the image resolution and batch size. Also, Splitting the dataset into training and validation datasets in 80:20 ratio
batch_size = 32
img_height = 180
img_width = 180

# Splitting the dataset into training and validation datasets
data_dir = tf.keras.utils.get_file(
    'flower_photos', 'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    untar=True)

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)


In [ ]:
# We will build a sequential CNN model using the following code:

num_classes = 5

model = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])


In [ ]:
#Compilethe modeland print model summary
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

img_path = '/path/to/flower/image.jpg'
img = image.load_img(img_path, target_size=(img_height, img_width))
img_array = image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print("This image most likely belongs to {} with a {:.2f} percent confidence."
      .format(class_names[np.argmax(score)], 100 * np.max(score)))
